In [3]:
import requests, json
import sqlite3
import pandas as pd

OLLAMA_URL = "http://localhost:11434"
OLLAMA_MODEL = "llama3.2:3b"

prompt = "Say 'ready' in one word."
r = requests.post(f"{OLLAMA_URL}/api/generate",
                  json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}, timeout=10)
r.status_code, r.json().get("response","")[:80]

(200, 'Ready.')

In [ ]:
import os 
os.chdir(os.getcwd().split('\\notebooks')[0])
main = os.getcwd()

DB_PATH = main + "\\data\\temporal\\exams.db"

In [ ]:
OLLAMA_OPTIONS = {"num_ctx": 1024, "num_gpu": 1}

In [5]:
con = sqlite3.connect(DB_PATH)
cols = [r[1] for r in con.execute("PRAGMA table_info(attempts);")]
if "reasons" not in cols:
    con.execute("ALTER TABLE attempts ADD COLUMN reasons TEXT;")
    con.commit()
con.close()
print("Schema OK")


Schema OK


In [6]:
con = sqlite3.connect(DB_PATH)
cur = con.cursor()

In [32]:
# add columns if missing
cols = [r[1] for r in cur.execute("PRAGMA table_info(attempts);")]
if "reasons" not in cols:
    cur.execute("ALTER TABLE attempts ADD COLUMN reasons TEXT;")
if "hint" not in cols:
    cur.execute("ALTER TABLE attempts ADD COLUMN hint TEXT;")
if "feedback_json" not in cols:
    cur.execute("ALTER TABLE attempts ADD COLUMN feedback_json TEXT;")

con.commit(); con.close()
print("Schema OK (attempts has reasons, hint, feedback_json).")

Schema OK (attempts has reasons, hint, feedback_json).


In [ ]:
def grade_with_local_llm(question:str, solution:str, student:str, timeout=30):
    """
    Calls Ollama locally to grade the student's answer.
    Returns a dict like the baseline grader.
    """
    prompt = f"""Grade the student's answer concisely.

Question: {question}
Reference solution: {solution}
Student answer: {student}

Return a JSON object with:
- score: a number between 0 and 1
- correct: true/false
- reasons: a short string
"""
    try:
        resp = requests.post(f"{OLLAMA_URL}/api/generate",
                             json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False},
                             timeout=timeout)
        txt = resp.json().get("response","").strip()
        data = json.loads(txt) if txt.startswith("{") else None
        if not data:
            return None
        # normalize output
        return {
            "score": float(data.get("score", 0)),
            "correct": bool(data.get("correct", False)),
            "cosine": None, "jaccard": None, "missing_keywords": [],
            "reasons": data.get("reasons","")
        }
    except Exception:
        return None


In [ ]:
def connect(): return sqlite3.connect(DB_PATH)

In [22]:
# (paste these small helper stubs or import them if you saved to a .py)
def get_user_id(username:str, db_path=DB_PATH) -> int:
    con = sqlite3.connect(db_path); cur = con.cursor()
    cur.execute("INSERT OR IGNORE INTO users(username) VALUES(?)", (username,))
    con.commit()
    cur.execute("SELECT user_id FROM users WHERE username=?", (username,))
    uid = cur.fetchone()[0]; con.close(); return uid

def pick_unseen(username:str, k:int=5, db_path=DB_PATH):
    uid = get_user_id(username, db_path)
    con = sqlite3.connect(db_path); cur = con.cursor()
    cur.execute("""
      SELECT q.exercise_id, q.topic_pred, e.date
      FROM questions q JOIN exams e ON q.exam_id=e.exam_id
      WHERE q.exercise_id NOT IN (SELECT exercise_id FROM attempts WHERE user_id=?)
      ORDER BY e.date ASC LIMIT ?""", (uid, k))
    rows = cur.fetchall(); con.close()
    return rows

def fetch_question(exercise_id:str, db_path=DB_PATH):
    con = sqlite3.connect(db_path); cur = con.cursor()
    cur.execute("""
      SELECT q.exercise_id, q.question, q.solution, q.topic_pred, e.date, e.exam_type
      FROM questions q JOIN exams e ON q.exam_id=e.exam_id
      WHERE q.exercise_id=?""", (exercise_id,))
    row = cur.fetchone(); con.close()
    return dict(zip(["exercise_id","question","solution","topic","date","exam_type"], row))


def _clean(s:str) -> str:
    s = (s or "").lower()
    s = re.sub(r"\(cid:\d+\)", " ", s); s = re.sub(r"[^a-z0-9\-\+\*/\^\=\(\)\[\]\{\}\., ]+"," ",s)
    return re.sub(r"\s+"," ", s).strip()
STOP = set("the a an and or of to for with from in on at is are be was were by as that this these those into over under if then else such".split())
def _keywords(s:str):
    toks = re.findall(r"[a-z0-9\^\+\-\*/=]+", _clean(s))
    return {t for t in toks if len(t)>=2 and t not in STOP}
def grade_answer(solution:str, student:str):
    sol = _clean(solution); ans = _clean(student)
    vec = TfidfVectorizer(analyzer="char_wb", ngram_range=(3,5), min_df=1)
    X = vec.fit_transform([sol, ans]); X = normalize(X)
    cos = float((X[0] @ X[1].T).A[0,0]) if X.shape[1] else 0.0
    Ks, Ka = _keywords(sol), _keywords(ans)
    jac = len(Ks & Ka) / max(1, len(Ks | Ka))
    score = 0.6*cos + 0.4*jac
    return {"score": round(score,4), "correct": score>=0.6, "cosine": round(cos,4), "jaccard": round(jac,4),
            "missing_keywords": list((Ks-Ka))[:8]}

def submit_answer_db(username:str, exercise_id:str, student_answer:str, db_path=DB_PATH):
    uid = get_user_id(username, db_path)
    q = fetch_question(exercise_id, db_path)
    g = grade_answer(q["solution"], student_answer)
    con = sqlite3.connect(db_path); cur = con.cursor()
    cur.execute("""INSERT INTO attempts(user_id, exercise_id, score, correct, cosine, jaccard, missing_keywords, student_answer)
                   VALUES(?,?,?,?,?,?,?,?)""",
                (uid, exercise_id, g["score"], int(g["correct"]), g["cosine"], g["jaccard"],
                 json.dumps(g["missing_keywords"]), student_answer))
    con.commit(); con.close()
    return {"exercise_id": exercise_id, "topic": q["topic"], **g}

def save_attempt(uid:int, exercise_id:str, g:dict, student_answer:str):
    con = connect(); cur = con.cursor()
    cur.execute("""
      INSERT INTO attempts(user_id, exercise_id, score, correct, cosine, jaccard, missing_keywords, student_answer)
      VALUES(?,?,?,?,?,?,?,?)
    """, (uid, exercise_id, g["score"], int(g["correct"]), g["cosine"], g["jaccard"],
          json.dumps(g.get("missing_keywords",[])), student_answer))
    con.commit(); con.close()

In [23]:
def submit_answer_llm_first(username:str, exercise_id:str, student_answer:str):
    q = fetch_question(exercise_id)  # your existing helper (DB-backed)
    # Try LLM
    g = grade_with_local_llm(q["question"], q["solution"], student_answer)
    # Fallback to baseline if LLM failed
    if not g:
        g = grade_answer(q["solution"], student_answer)  # your baseline grader
    # Save
    _uid = get_user_id(username)
    save_attempt(_uid, exercise_id, g, student_answer)
    return g


In [24]:
res = submit_answer_llm_first("student1", "Exercise 1", "My attempt about contraction mapping ")
res

{'score': 0.0,
 'correct': False,
 'cosine': None,
 'jaccard': None,
 'missing_keywords': [],
 'reasons': "The student's response appears to be unrelated to the Riesz representation theorem. They mentioned contraction mapping, which is a concept from metric spaces and has no direct connection to the characterization of linear functionals."}

# LLM FEEDBACK

In [26]:
import requests, json, re

def llm_grade_and_feedback(question:str, solution:str, student:str, timeout=60):
    """
    Ask the local LLM for a compact judgment + 1-line hint.
    Returns dict or None on failure.
    """
    prompt = f"""You grade short math answers. Be brief and avoid giving the full solution.

Question:
{question}

Reference solution:
{solution}

Student answer:
{student}

Return ONLY a compact JSON with exactly these keys:
- "score": number 0..1
- "correct": true/false
- "explanation": one short sentence explaining the verdict
- "hint": one short hint the student can try next (DO NOT reveal the full solution)
"""
    try:
        r = requests.post(f"{OLLAMA_URL}/api/generate",
                          json={"model": OLLAMA_MODEL, "prompt": prompt,
                                "stream": False, "format":"json", "options": OLLAMA_OPTIONS},
                          timeout=timeout)
        data = json.loads(r.json().get("response","").strip())
        # normalize
        return {
            "score": float(data.get("score", 0)),
            "correct": bool(data.get("correct", False)),
            "reasons": data.get("explanation",""),
            "hint": data.get("hint",""),
            "cosine": None, "jaccard": None, "missing_keywords": []
        }
    except Exception:
        return None


# SAVE ATTEMPT WITH FEEDBACK

In [27]:
def grade_best_with_feedback(question:str, solution:str, student:str):
    g = llm_grade_and_feedback(question, solution, student)
    if g: 
        return g
    # fallback to baseline if LLM fails
    b = grade_answer(solution, student)  # <- your existing baseline function
    b["reasons"] = "Similarity-based baseline verdict."
    b["hint"] = ""
    return b


In [28]:
import sqlite3, json

def save_attempt(uid:int, exercise_id:str, g:dict, student_answer:str, db_path=DB_PATH):
    con = sqlite3.connect(db_path); cur = con.cursor()
    cur.execute("""
      INSERT INTO attempts(user_id, exercise_id, score, correct, cosine, jaccard,
                           missing_keywords, student_answer, reasons, hint, feedback_json)
      VALUES(?,?,?,?,?,?,?,?,?,?,?)
    """, (uid, exercise_id,
          g.get("score",0.0), int(bool(g.get("correct",False))),
          g.get("cosine"), g.get("jaccard"),
          json.dumps(g.get("missing_keywords",[])),
          student_answer, g.get("reasons",""), g.get("hint",""),
          json.dumps({k:v for k,v in g.items() if k not in ("cosine","jaccard","missing_keywords")})
         ))
    con.commit(); con.close()

def submit_answer_with_feedback(username:str, exercise_id:str, student_answer:str):
    uid = get_user_id(username)
    q = fetch_question(exercise_id)
    g = grade_best_with_feedback(q["question"], q["solution"], student_answer)
    save_attempt(uid, exercise_id, g, student_answer)
    return {"exercise_id": exercise_id, "topic": q["topic"], "date": q["date"], **g}


# Next Question (Recommender)

In [29]:
def recommend_next(username:str, k:int=5, db_path=DB_PATH):
    uid = get_user_id(username, db_path)
    con = sqlite3.connect(db_path)

    # unseen pool
    unseen = pd.read_sql("""
      SELECT q.exercise_id, q.topic_pred AS topic
      FROM questions q
      WHERE q.exercise_id NOT IN (SELECT exercise_id FROM attempts WHERE user_id=?)
    """, con, params=(uid,))

    # recent mistakes to review (last attempt was incorrect)
    mistakes = pd.read_sql("""
      SELECT a.exercise_id, q.topic_pred AS topic, MAX(a.ts) as last_ts
      FROM attempts a JOIN questions q ON a.exercise_id=q.exercise_id
      WHERE a.user_id=? AND a.correct=0
      GROUP BY a.exercise_id, q.topic_pred
      ORDER BY last_ts DESC
      LIMIT 10
    """, con, params=(uid,))

    # weak topics by avg score
    perf = pd.read_sql("""
      SELECT q.topic_pred AS topic, AVG(a.score) AS avg_score
      FROM attempts a JOIN questions q ON a.exercise_id=q.exercise_id
      WHERE a.user_id=?
      GROUP BY q.topic_pred
      ORDER BY avg_score ASC
    """, con, params=(uid,))
    con.close()

    picks = []

    # 1) take up to ceil(k*0.4) recent mistakes to review
    n_rev = max(1, int(0.4*k))
    if not mistakes.empty:
        picks += mistakes["exercise_id"].tolist()[:n_rev]

    # 2) fill remaining with unseen, prioritizing weak topics
    remaining = k - len(picks)
    if remaining > 0 and not unseen.empty:
        weak = set(perf["topic"].head(max(1, len(perf)//2)).tolist()) if not perf.empty else set()
        unseen_weak = unseen[unseen["topic"].isin(weak)]
        pool = unseen_weak if not unseen_weak.empty else unseen
        picks += pool.sample(min(remaining, len(pool)), random_state=42)["exercise_id"].tolist()

    # de-dup just in case
    seen = set(); ordered=[]
    for x in picks:
        if x not in seen:
            ordered.append(x); seen.add(x)
    return ordered[:k]


# Quick test

In [13]:
USERNAME = "student1"

# pick any unseen question (or choose one ID manually)
rows = pick_unseen(USERNAME, k=1)
if not rows:
    print("No unseen questions left – try resetting attempts or loading more data.")
else:
    qid = rows[0][0]
    q = fetch_question(qid)
    print("Question:", q["exercise_id"], "| Topic:", q["topic"], "| Date:", q["date"])
    print(q["question"], "\n")

    # simulate an answer:
    res = submit_answer_llm_first(USERNAME, qid, "Outline the contraction mapping argument and fixed point uniqueness...")
    res


Question: Exercise 3 | Topic: brouwer_fixed_point_theorem | Date: 2025-08-29
Prove that a linear operator T : V V between normed vector spaces is continuous at 1 2 ! a point v V if and only if it is continuous on V . 1 1 2 



In [33]:
USERNAME = "student1"

# pick or recommend
cand = recommend_next(USERNAME, k=3)
print("Recommendations:", cand)

if not cand:
    # fall back to any unseen
    rows = pick_unseen(USERNAME, k=1)
    cand = [rows[0][0]] if rows else []

if cand:
    ex_id = cand[0]
    q = fetch_question(ex_id)
    print(f"\nQuestion {q['exercise_id']} · {q['topic']} · {q['date']}\n{q['question']}\n")
    # simulate student answer
    res = submit_answer_with_feedback(USERNAME, ex_id, "I think we use a contraction and apply Banach to get the fixed point...")
    print("Verdict:", "✅" if res["correct"] else "❌", "| Score:", res["score"])
    print("Why:", res["reasons"])
    print("Hint:", res["hint"])


Recommendations: ['Exercise 1', 'Exercise 4']

Question Exercise 1 · linear_functionals_and_operators · 2025-08-29
Consider a functional f : R n R. State and prove the Riesz representation theorem ! (that is, the theorem that provides a characterization for linear functionals).

Verdict: ❌ | Score: 0.2
Why: Incorrect approach
Hint: Focus on Riesz's original paper for inspiration
